In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
from midi2audio import FluidSynth

In [8]:
fs = FluidSynth()
fs.midi_to_audio('test.mid', 'truc.wav')

FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

Rendering audio to file 'truc.wav'..


fluidsynth: warning: No preset found on channel 9 [bank=128 prog=0]
fluidsynth: panic: An error occurred while reading from stdin.


In [3]:
pm = pretty_midi.PrettyMIDI('test.mid')

In [4]:
pm.fluidsynth()

fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable


array([ 0.        ,  0.        ,  0.        , ..., -0.00043497,
        0.        ,  0.        ])

In [19]:
import numpy as np
import pretty_midi
import visual_midi
import joblib
from MIDIComposingAI.utils import piano_roll_to_pretty_midi
from MIDIComposingAI.create_dataset import *
from MIDIComposingAI.get_back_data import *
import pickle

In [34]:
from midi2audio import FluidSynth

In [5]:
import subprocess
subprocess.call(["fluidsynth", "-V"])

FluidSynth runtime version 2.1.1
Copyright (C) 2000-2020 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of E-mu Systems, Inc.

FluidSynth executable version 2.1.1
Sample type=double


0

In [35]:
FluidSynth().midi_to_audio('test.mid', 'audio.wav')

FileNotFoundError: [Errno 2] No such file or directory: 'fluidsynth'

In [3]:
p = pretty_midi.PrettyMIDI('../raw_data/personal.mid')

In [4]:
tree = joblib.load('../Models/API_tree.joblib')

In [5]:
zeros = np.zeros((128, 750))

In [6]:
pers = p.get_piano_roll(fs=50)

def reshape_piano_roll(piano_roll, size=500):
    """
    Reshape the piano roll to match into the required shape for predictions
    """
    if piano_roll.shape[-1] < size:
        
        zeros_array = np.zeros((128, size - piano_roll.shape[-1]))
        return np.concatenate((piano_roll, zeros_array), axis=1)
    
    if piano_roll.shape[-1] > size:
        
        piano_rolls =  np.split(piano_roll, [500], axis=1)
        
        while piano_rolls[-1].shape[-1] > 500:
            
            last_split = np.split(piano_rolls[-1], [500], axis=1)
            piano_rolls[-1] = last_split[0]
            piano_rolls.append(last_split[-1])
        
        # while np.array(piano_rolls[-1]).shape > size:
        #     piano_rolls.append(np.split(piano_roll[-1], [500], axis=1))
        return piano_rolls

In [7]:
zeros = np.zeros((128, 3500))

In [8]:
pers = reshape_piano_roll(pers)

In [9]:
chords = adding_chords_info('../raw_data/chords_midi.csv', pers.reshape(1, 128, 500))

In [10]:
pers_reshaped = pers.reshape(1, -1)
pers_reshaped = np.concatenate((chords, pers_reshaped), axis=1) 

In [11]:
pred = tree.predict(pers_reshaped)

In [22]:
pred_mel = [assembled_target_to_melody(p) for p in pred]

In [24]:
pred_mel[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
copy = pers.copy()

In [29]:
full_music = assemblate_accompaniment_melody(copy, pred_mel[0])

In [33]:
piano_roll_to_pretty_midi(full_music, fs=50).write('test.mp3')

In [30]:
%%capture --no-display

# Plot a piano roll visulasiation of the file
coloring = visual_midi.presets.Coloring.INSTRUMENT
plotter = visual_midi.Plotter(coloring=coloring)
plotter.show_notebook(piano_roll_to_pretty_midi(full_music, fs=50))

Loading BokehJS ...

Column(id='1555', ...)

In [5]:
def extract_accompaniment_melody(pretty_midi_file, fs=50, sample_length=10, sample_set=0):
    """
    Extract melody from a pretty_midi file.
    
    Args :
        pretty_midi_file : a pretty_midi.Pretty_midi() file
        fs : number of frame per second, use to create the piano roll from the pretty_midi file
        ratio : the ratio above wich we won't accept a note to be extract, it prevents from having jerky melodies
        sample_lentgh : integer, the length in seconds of the sample we wan't to extract the melody from.
        sample_set : integer, it allows you to choose where in the piece you want to extract the melody
                      
    Return : a tuple of pretty_midi.piano_roll variables : (accompaniment, melody)
    """
    # and (abs(last_played_note - piano_roll[j][i])/nb_instant > ratio)

    sample_size = sample_length * fs
    
    piano_roll = pretty_midi_file.get_piano_roll(fs=fs)[:, sample_size*sample_set:sample_size*(sample_set+1)]
    empty_piano_roll = np.zeros(piano_roll.shape)

    liste = []
    # nb_instant = 0
    for i in range(sample_size):
        # nb_instant += 1
        for j in range(127, 0, -1):
            try:
                if piano_roll[j][i] > 0:
                    last_played_note = piano_roll[j][i]
                    # We want our values to be between 0 and 127
                    # if last_played_note <= 127:
                    empty_piano_roll[j][i] = last_played_note
                    # else:
                    #     empty_piano_roll[j][i] = 127
                    piano_roll[j][i] = 0
                    liste.append([[j],[i]])
                    # nb_instant = 0
                    break
            except:
                if piano_roll[j][i] > 0:
                    last_played_note = piano_roll[j][i]
                    if last_played_note <= 127:
                        empty_piano_roll[j][i] = last_played_note
                    else:
                        empty_piano_roll[j][i] = 127
                    piano_roll[j][i] = 0
                    nb_instant = 0
                    break
    return (piano_roll, empty_piano_roll)

def separate_pitch_velocity(target):
    """
    Separate pitch and velocity within the target
    """
    # Lists of each velocities and pitches for each sample
    sample_velocities = []
    sample_pitches = []
    
    for sample in target:
        # Lists of velocities and pitches within the sample
        velocities = []
        pitches = []
        
        for frame in sample.T:
            frame = list(frame)
            velocity = np.sum(frame)
            velocities.append(velocity)
            pitches.append(frame.index(velocity))
        sample_velocities.append(velocities)
        sample_pitches.append(pitches)
    
    return (sample_pitches, sample_velocities)

def create_sample(pretty_midi_file, fs):
    """
    Return a sample of the file
    """
    piano_roll = pretty_midi_file.piano_roll(fs=fs)
    

def create_simple_dataset(file, mode=None):
    """
    Create a simple dataset for ML/DL

    Args :
        file : A pretty_midi file

    Return : A tuple with X = accompaniment, y = melody
    """
    X = []
    y = []
    i = 0
    while True:
        try:
            accompaniment, melody = extract_accompaniment_melody(file, sample_set=i)
            X.append(accompaniment)
            y.append(melody)
            i += 1
        except:
            break

    return np.array(X), np.array(y)

def create_nparray_dataset(file, directory ,name, store=True):
    """
    Create a nparray dataset
    """
    X, y = create_simple_dataset(file)
    
    pitches, velocities = separate_pitch_velocity(y)
    
    i = 0

    y_melody = np.array(
                [(pitch, velocity) for pitch, velocity in zip(pitches, velocities)]
            )

    # X_accompaniment = np.array([accompaniment.T for accompaniment in X])
        
    # Then we add the two target to the dataframe
    # y_pitch = np.array([np.array(pitch) for pitch in pitches])
    # y_velocity = np.array([np.array(velocity) for velocity in velocities])

    dataset = (X, y_melody)
    
    if store:
        joblib.dump(dataset, f'../raw_data/pandas_dataframes/{directory}/{name}')
    else:
        return dataset
    
    # In the end we need to delete the variables in order to save some RAM
    del([X, y, y_melody, pitches, velocities, dataset])

def create_tuple_target_dataset(file):
    """
    Create a dataset with a target being a list of tuples (pitch, velocity)
    Args:
        file : a pretty_midi object/file
    """
    X, y = create_simple_dataset(file)

    new_target = []

    for sample in y:

        frames = []

        for frame in sample.T:

            list_of_tuples = [(note, velocity) if velocity > 0 else (0, 0) for note, velocity in enumerate(frame)]
            frames.append(list_of_tuples)

        new_target.append(frames)
        
    return X, np.array(new_target)

def create_classified_melody(melody, to_list=True):
    """
    Return a classified melody (0 if no note is played, 1 if a note is played)
    """
    classified_melody = [1 if note > 0 else 0 for note in melody]

    if not to_list:
        return np.array(classified_melody)
    
    return classified_melody
